In [ ]:
import LGBMClassifier as lgb 

lgb_model = lgb.LGBMClassifier(bagging_fraction=0.7, bagging_freq=2, boosting_type='gbdt', 
class_weight=None, colsample_bytree=1.0, feature_fraction=0.8, 
importance_type='split', learning_rate=0.0005, max_depth=-1, 
min_child_samples=1, min_child_weight=0.001, min_split_gain=0.2, 
n_estimators=30, n_jobs=-1, num_leaves=256, objective=None, 
random_state=123, reg_alpha=3, reg_lambda=0.001, silent='warn', 
subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
